在这个notebook中，我们

- 已知：高低频模型预测结果，当前仓位
- 目标：输出仓位变动，报单价格

这里的代码做为返回策略线程的基础。

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time

pd.options.mode.chained_assignment = None

# 高频部分

In [2]:
leverage = 1.8  # 杠杆率
holding = 10  # 持有周期
num_newpos = 25  # 每轮分别新做多和做空股数

capital = 2e8
K = capital * leverage  # 可操作资金

In [3]:
# 每个tick拿到数据计算因子 --> 放入模型预测 --> 返回策略输出

In [4]:
model = pickle.load(open('Models/MLP_SOIR.sav', 'rb')) # 使用的模型

In [5]:
track_newpos = np.zeros([holding, 500]) # keep track of newly initiatied positions from the previous 10 ticks

newdata = np.random.rand(500, 48) # 新拿到的数据 (newdata[:,8] = bid1_price, newdata[:,28] = ask1_price)
alphas = np.random.rand(500, 10)  # 计算出的因子
current_pos = np.random.rand(500) # 当前仓位

In [6]:
pred = model.predict(alphas) # shape (500)

In [7]:
longstock = pred.argsort()[-num_newpos:] # 做多哪些股票
shortstock = pred.argsort()[:num_newpos] # 做空哪些股票

newpos = np.zeros(500)

afford = capital*leverage/(2*num_newpos)/holding/newdata[:,5]

newpos[longstock] = afford[longstock]
newpos[shortstock] = -afford[shortstock]

track_newpos = np.append(track_newpos, np.array([newpos]), axis = 0)[1:]

obj_pos = track_newpos.sum(axis = 0)

submit_pos = obj_pos - current_pos  # 提交仓位变化

bid1 = newdata[:,8]
ask1 = newdata[:,28]

# 做多报卖盘最优价（ask1），做空报买盘最优价（bid1）

price = ask1 * (submit_pos > 0) + bid1 * (submit_pos < 0)  # 提交报单价格